<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a>
</p>
<p align="center">
<a href="https://discord.gg/AMApC2UzVY"><img alt="Discord" src="https://img.shields.io/badge/discord-chat-purple?color=%235765F2&label=discord&logo=discord"></a>
<a href="https://twitter.com/vlmrun"><img alt="Twitter Follow" src="https://img.shields.io/twitter/follow/vlmrun.svg?style=social&logo=twitter"></a>
</p>
</div>

Welcome to **[VLM Run Cookbooks](https://github.com/vlm-run/vlmrun-cookbook)**, a comprehensive collection of examples and notebooks demonstrating the power of structured visual understanding using the [VLM Run Platform](https://app.vlm.run).


## Case Study: Object Detection with VLM Run (Node.js)

This notebook demonstrates how to use VLM Run's detection capabilities to identify and locate objects, people, and faces in images using **Node.js/TypeScript**. We'll cover:

- **Object Detection**: Detect 80+ COCO dataset classes (person, car, cat, etc.)
- **Person Detection**: Specialized detection for people in images
- **Face Detection**: Detect and locate faces with high precision

All detections return bounding boxes in normalized `xywh` format (x, y, width, height) where values are between 0-1, along with labels.

Reference: [VLM Run Detection Documentation](https://docs.vlm.run/agents/capabilities/image/detection)


### Environment Setup

To get started, install the required packages and sign up for an API key on the [VLM Run App](https://app.vlm.run).
- Store the VLM Run API key under the `VLMRUN_API_KEY` environment variable.


## Prerequisites

* Node.js 18+
* VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
* Deno or tslab kernel for running TypeScript in Jupyter


## Setup

First, let's install the required packages:


In [ ]:
// If using Deno kernel, install dependencies via npm specifiers
// For tslab, run: npm install openai zod zod-to-json-schema in your project directory


In [1]:
import OpenAI from "openai";
import { z } from "zod";
import { zodToJsonSchema } from "npm:zod-to-json-schema";


In [2]:
// Get API key from environment variable
const VLMRUN_API_KEY = "7R8aQ5XlH00LlGdlpe_SLywfsAwszz";

if (!VLMRUN_API_KEY) {
    throw new Error("Please set the VLMRUN_API_KEY environment variable");
}

console.log("✓ API Key loaded successfully");


✓ API Key loaded successfully


Let's initialize the OpenAI client configured for VLM Run


In [3]:
// Initialize the OpenAI client with VLM Run's Agent API
const client = new OpenAI({
    baseURL: "https://agent.vlm.run/v1/openai",
    apiKey: VLMRUN_API_KEY
});

console.log("✓ VLM Run client initialized!");


✓ VLM Run client initialized!


## Define Detection Schemas

We'll define Zod schemas for the detection responses (TypeScript equivalent of Python's Pydantic):


In [4]:
// Define the Detection schema using Zod
const DetectionSchema = z.object({
    label: z.string().describe("Name of the detected object"),
    xywh: z.tuple([z.number(), z.number(), z.number(), z.number()])
        .describe("Bounding box (x, y, width, height) - normalized values 0-1")
});

// Define the Detections response schema
const DetectionsSchema = z.object({
    detections: z.array(DetectionSchema).describe("List of detected objects")
});

// Type inference from schemas
type Detection = z.infer<typeof DetectionSchema>;
type Detections = z.infer<typeof DetectionsSchema>;

// Convert Zod schema to JSON Schema for API
const detectionsJsonSchema = zodToJsonSchema(DetectionsSchema);
console.log("✓ Detection schemas defined");
console.log("JSON Schema:", JSON.stringify(detectionsJsonSchema, null, 2));


✓ Detection schemas defined
JSON Schema: {
  "type": "object",
  "properties": {
    "detections": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "label": {
            "type": "string",
            "description": "Name of the detected object"
          },
          "xywh": {
            "type": "array",
            "minItems": 4,
            "maxItems": 4,
            "items": [
              {
                "type": "number"
              },
              {
                "type": "number"
              },
              {
                "type": "number"
              },
              {
                "type": "number"
              }
            ],
            "description": "Bounding box (x, y, width, height) - normalized values 0-1"
          }
        },
        "required": [
          "label",
          "xywh"
        ],
        "additionalProperties": false
      },
      "description": "List of detected objects"
    }
  }

## Helper Functions

Let's create utility functions for detection and displaying results:


In [5]:
/**
 * Perform object detection on an image using VLM Run's Agent API.
 * 
 * @param imageUrl - URL of the image to analyze
 * @param prompt - Detection prompt (e.g., "Detect all objects", "Detect all people")
 * @returns Detections object with bounding boxes
 */
async function detectObjects(imageUrl: string, prompt: string = "Detect all objects in this image"): Promise<Detections> {
    const response = await client.chat.completions.create({
        model: "vlm-agent-1",
        messages: [
            {
                role: "user",
                content: [
                    { type: "text", text: prompt },
                    { type: "image_url", image_url: { url: imageUrl, detail: "auto" } }
                ]
            }
        ],
        // Use VLM Run's format - schema at top level
        response_format: { 
            type: "json_schema", 
            schema: detectionsJsonSchema
        } as any
    });
    
    const rawContent = response.choices[0].message.content || "{}";
    const parsed = JSON.parse(rawContent);
    return DetectionsSchema.parse(parsed);
}

/**
 * Format detection results for display.
 * 
 * @param detections - Array of Detection objects
 * @param imageWidth - Width of image in pixels (for coordinate display)
 * @param imageHeight - Height of image in pixels (for coordinate display)
 */
function formatDetections(detections: Detection[], imageWidth?: number, imageHeight?: number): void {
    console.log(`\n📦 Found ${detections.length} objects:\n`);
    
    detections.forEach((det, i) => {
        const [x, y, w, h] = det.xywh;
        
        if (imageWidth && imageHeight) {
            // Convert normalized coordinates to pixel coordinates
            const x_px = Math.round(x * imageWidth);
            const y_px = Math.round(y * imageHeight);
            const w_px = Math.round(w * imageWidth);
            const h_px = Math.round(h * imageHeight);
            console.log(`  ${i + 1}. ${det.label}`);
            console.log(`     Normalized: x=${x.toFixed(3)}, y=${y.toFixed(3)}, w=${w.toFixed(3)}, h=${h.toFixed(3)}`);
            console.log(`     Pixels:     x=${x_px}, y=${y_px}, w=${w_px}, h=${h_px}`);
        } else {
            console.log(`  ${i + 1}. ${det.label}: xywh=[${x.toFixed(3)}, ${y.toFixed(3)}, ${w.toFixed(3)}, ${h.toFixed(3)}]`);
        }
    });
}

/**
 * Group detections by label and count them.
 * 
 * @param detections - Array of Detection objects
 * @returns Object with label counts
 */
function countByLabel(detections: Detection[]): Record<string, number> {
    return detections.reduce((acc, det) => {
        acc[det.label] = (acc[det.label] || 0) + 1;
        return acc;
    }, {} as Record<string, number>);
}

console.log("✓ Helper functions defined");


✓ Helper functions defined


## Example 1: Person Detection

Let's detect people in an image of a group.


In [6]:
// Image with people
const peopleImageUrl = "https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800";

console.log("📷 Image URL:", peopleImageUrl);
console.log("\nDetecting people in the image...\n");

// Perform person detection
const personResult = await detectObjects(peopleImageUrl, "Detect all people in this image");

// Display results
console.log(`Found ${personResult.detections.length} people in the image:`);
personResult.detections.forEach((det, i) => {
    console.log(`  Person ${i + 1}: ${det.label}, xywh=[${det.xywh.map(v => v.toFixed(3)).join(", ")}]`);
});

// Show summary
const labelCounts = countByLabel(personResult.detections);
console.log("\n📊 Detection Summary:");
console.table(labelCounts);


📷 Image URL: https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800

Detecting people in the image...

Found 4 people in the image:
  Person 1: person, xywh=[0.325, 0.266, 0.125, 0.619]
  Person 2: person, xywh=[0.400, 0.271, 0.148, 0.619]
  Person 3: person, xywh=[0.487, 0.271, 0.144, 0.619]
  Person 4: person, xywh=[0.592, 0.237, 0.113, 0.648]

📊 Detection Summary:
┌────────┬────────┐
│ (idx)  │ Values │
├────────┼────────┤
│ person │      4 │
└────────┴────────┘


## Example 2: General Object Detection

Let's detect various objects in a street scene.


In [7]:
// Street scene image (crossroad with vehicles and pedestrians)
const streetImageUrl = "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg";

console.log("📷 Image URL:", streetImageUrl);
console.log("\nDetecting all objects in the street scene...\n");

// Perform general object detection
const objectResult = await detectObjects(streetImageUrl, "Detect all objects in this image including people, vehicles, and other objects");

// Format and display results with assumed image dimensions
formatDetections(objectResult.detections, 800, 600);

// Show counts by category
const objectCounts = countByLabel(objectResult.detections);
console.log("\n📊 Objects Detected by Category:");
console.table(objectCounts);


📷 Image URL: https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg

Detecting all objects in the street scene...


📦 Found 161 objects:

  1. car
     Normalized: x=0.000, y=0.445, w=0.145, h=0.252
     Pixels:     x=0, y=267, w=116, h=151
  2. car
     Normalized: x=0.219, y=0.408, w=0.183, h=0.312
     Pixels:     x=175, y=245, w=146, h=187
  3. car
     Normalized: x=0.478, y=0.432, w=0.107, h=0.073
     Pixels:     x=382, y=259, w=86, h=44
  4. car
     Normalized: x=0.686, y=0.456, w=0.077, h=0.075
     Pixels:     x=549, y=274, w=62, h=45
  5. car
     Normalized: x=0.756, y=0.442, w=0.080, h=0.115
     Pixels:     x=605, y=265, w=64, h=69
  6. car
     Normalized: x=0.820, y=0.436, w=0.088, h=0.148
     Pixels:     x=656, y=262, w=70, h=89
  7. car
     Normalized: x=0.897, y=0.430, w=0.103, h=0.196
     Pixels:     x=718, y=258, w=82, h=118
  8. car
     Normalized: x=0.585, y=0.427, w=0.043, h=0.037
     Pixels:     x=468, y=256,

## Example 3: Face Detection

Let's specifically detect faces in an image.


In [8]:
// Use the same people image for face detection
console.log("📷 Image URL:", peopleImageUrl);
console.log("\nDetecting faces in the image...\n");

// Perform face detection
const faceResult = await detectObjects(peopleImageUrl, "Detect all faces in this image");

// Display face detection results
console.log(`Found ${faceResult.detections.length} faces:`);
faceResult.detections.forEach((det, i) => {
    const [x, y, w, h] = det.xywh;
    console.log(`  Face ${i + 1}: x=${x.toFixed(3)}, y=${y.toFixed(3)}, w=${w.toFixed(3)}, h=${h.toFixed(3)}`);
});


📷 Image URL: https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800

Detecting faces in the image...

Found 4 faces:
  Face 1: x=0.324, y=0.266, w=0.118, h=0.622
  Face 2: x=0.405, y=0.271, w=0.137, h=0.620
  Face 3: x=0.491, y=0.273, w=0.144, h=0.622
  Face 4: x=0.589, y=0.240, w=0.117, h=0.648


## Example 4: Batch Processing Multiple Images

Process multiple images and aggregate detection results.


In [9]:
// Multiple images for batch processing
const batchImages = [
    { url: "https://storage.googleapis.com/vlm-data-public-prod/hub/examples/image.object-detection/crossroad.jpg", name: "Street Scene" },
    { url: "https://images.unsplash.com/photo-1511632765486-a01980e01a18?w=800", name: "Group of People" },
    { url: "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400", name: "Cat" },
];

interface BatchDetectionResult {
    imageName: string;
    url: string;
    objectCount: number;
    labels: string[];
    error?: string;
}

async function processBatch(images: { url: string; name: string }[]): Promise<BatchDetectionResult[]> {
    const results: BatchDetectionResult[] = [];
    
    for (const img of images) {
        try {
            console.log(`Processing: ${img.name}...`);
            const detections = await detectObjects(img.url);
            const uniqueLabels = [...new Set(detections.detections.map(d => d.label))];
            
            results.push({
                imageName: img.name,
                url: img.url,
                objectCount: detections.detections.length,
                labels: uniqueLabels
            });
        } catch (error) {
            console.log(`Error processing ${img.name}: ${error}`);
            results.push({
                imageName: img.name,
                url: img.url,
                objectCount: 0,
                labels: [],
                error: String(error)
            });
        }
    }
    
    return results;
}

// Process all images
const batchResults = await processBatch(batchImages);

// Display batch results
console.log("\n=== Batch Detection Results ===\n");
console.table(batchResults.map(r => ({
    Image: r.imageName,
    "Objects Found": r.objectCount,
    "Unique Labels": r.labels.join(", ") || "Error"
})));


Processing: Street Scene...
Processing: Group of People...
Processing: Cat...

=== Batch Detection Results ===

┌───────┬───────────────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────┐
│ (idx) │ Image             │ Objects Found │ Unique Labels                                                                   │
├───────┼───────────────────┼───────────────┼─────────────────────────────────────────────────────────────────────────────────┤
│     0 │ "Street Scene"    │            82 │ "person, taxi, car, taxi roof sign, license plate, sign, Office, traffic light" │
│     1 │ "Group of People" │             4 │ "person"                                                                        │
│     2 │ "Cat"             │             3 │ "cat, green background, wooden surface"                                         │
└───────┴───────────────────┴───────────────┴───────────────────────────────────────────────────────────────────────────

## Understanding Bounding Box Format

VLM Run returns bounding boxes in **normalized xywh format**:

| Field | Description | Range |
|-------|-------------|-------|
| `x` | Left edge of bounding box | 0.0 - 1.0 |
| `y` | Top edge of bounding box | 0.0 - 1.0 |
| `w` | Width of bounding box | 0.0 - 1.0 |
| `h` | Height of bounding box | 0.0 - 1.0 |

To convert to pixel coordinates, multiply by image dimensions:
```typescript
const x_px = x * imageWidth;
const y_px = y * imageHeight;
const w_px = w * imageWidth;
const h_px = h * imageHeight;
```


## Best Practices & Tips

### Detection Quality
- Use specific prompts for better results (e.g., "Detect all cars" vs "Detect objects")
- For crowded scenes, consider detecting specific object types separately
- Use higher resolution images for better small object detection

### Supported Object Classes
VLM Run can detect 80+ COCO dataset classes including:
- **People**: person, face
- **Vehicles**: car, truck, bus, motorcycle, bicycle, airplane, boat
- **Animals**: cat, dog, bird, horse, cow, sheep
- **Objects**: chair, table, laptop, phone, book, bottle, cup

### Performance Tips
- Batch similar detection tasks for efficiency
- Cache detection results for frequently analyzed images
- Use appropriate image sizes (800-1200px width is usually sufficient)


## Additional Resources

- [VLM Run Documentation](https://docs.vlm.run)
- [Detection API Reference](https://docs.vlm.run/agents/capabilities/image/detection)
- [VLM Run Node.js SDK](https://github.com/vlm-run/vlmrun-node-sdk)
- [More Examples](https://github.com/vlm-run/vlmrun-cookbook)
- [Discord Community](https://discord.gg/AMApC2UzVY)
